## Explore Board Representations

In [1]:
import numpy as np

## Ideas
* cribbing this from [pgx](https://github.com/sotetsuk/pgx/blob/main/docs/backgammon.md)
* there are 24 points, you represent pieces by 1 for white and -1 for black
* you add to the point you move to, you subtract to the point you take away
* you evaluate legal moves by checking to see that >= -1 for white, <= 1 for black
* say white goes 0-23, black goes 23-0

In [2]:
board = np.zeros([1,24],dtype=int)

In [3]:
board

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [4]:
board.reshape([4,6])

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [5]:
def create_initial_board():
    'intialize a board array with the starting positions of all checkers'
    initial_board = np.array([2, 0, 0, 0, 0, -5,
                       0, -3, 0, 0, 0, 5,
                       -5, 0, 0, 0, 3, 0,
                       5, 0, 0, 0, 0, -2])
    return initial_board

def roll():
    return np.random.randint(low=1, high=6, size=2)

In [6]:
initial_board = create_initial_board()

In [7]:
initial_board

array([ 2,  0,  0,  0,  0, -5,  0, -3,  0,  0,  0,  5, -5,  0,  0,  0,  3,
        0,  5,  0,  0,  0,  0, -2])

In [8]:
test_roll = roll()
test_roll

array([3, 4])

## Translate a 24 value numpy array into a visualized board state 

In [9]:
def create_checkers(position):
    drawn_points = []
    for point in position[:12]:
        if point == 0:
            drawn_point = 10 * ' '
        if point >= 0:
            drawn_point = 'X' * point + (10 - point) * ' '
        if point < 0:
            drawn_point = 'O' * abs(point) + (10 + point) * ' '
        drawn_points.append(drawn_point)
    for point in position[12:]:
        if point == 0:
            drawn_point = 10 * ' '
        if point >= 0:
            drawn_point = (10 - point) * ' ' + 'X' * point 
        if point < 0:
            drawn_point = (10 + point) * ' ' + 'O' * abs(point)  
        drawn_points.append(drawn_point)
    return drawn_points

def calculate_remaining_pips(position):
    x_pips = o_pips = 0
    for i, point in enumerate(position):
        if point > 0:
            dist = 23 - i
            x_pips += point * dist
        if point < 0:
            dist = i
            o_pips += point * dist * -1
    return x_pips, o_pips
    
def draw_board(position):
    drawn_position = create_checkers(position) 
    x_pips, o_pips = calculate_remaining_pips(position)
    
    print(f'''
     |---------------------|
  12 |{drawn_position[11]}|{drawn_position[12]}|13
  11 |{drawn_position[10]}|{drawn_position[13]}|14
  10 |{drawn_position[9]}|{drawn_position[14]}|15
   9 |{drawn_position[8]}|{drawn_position[15]}|16
   8 |{drawn_position[7]}|{drawn_position[16]}|17
   7 |{drawn_position[6]}|{drawn_position[17]}|18
     |---------------------|
     |---------------------|
   6 |{drawn_position[5]}|{drawn_position[18]}|19
   5 |{drawn_position[4]}|{drawn_position[19]}|20
   4 |{drawn_position[3]}|{drawn_position[20]}|21
   3 |{drawn_position[2]}|{drawn_position[21]}|22
   2 |{drawn_position[1]}|{drawn_position[22]}|23
   1 |{drawn_position[0]}|{drawn_position[23]}|24
     |---------------------|

    X remaining pips : {x_pips}
    O remaining pips : {o_pips}
    ''')

In [10]:
draw_board(initial_board)


     |---------------------|
  12 |XXXXX     |     OOOOO|13
  11 |          |          |14
  10 |          |          |15
   9 |          |          |16
   8 |OOO       |       XXX|17
   7 |          |          |18
     |---------------------|
     |---------------------|
   6 |OOOOO     |     XXXXX|19
   5 |          |          |20
   4 |          |          |21
   3 |          |          |22
   2 |          |          |23
   1 |XX        |        OO|24
     |---------------------|

    X remaining pips : 152
    O remaining pips : 152
    


## TODO
- [x] make function to update board from move
- [x] make it work for both colors
    * try to refactor it to be simpler, it looks terrible
- [x] make moves for both sides go down from 24 to 0
    - perhaps invert the position and multiply all values by -1
- [ ] update position function to handle hits
- [ ] add in the bar

In [58]:
def parse_move(move):
    first_move = move.split(' ')[0]
    second_move = move.split(' ')[1]
    
    first_start = int(first_move.split('/')[0])
    first_end = int(first_move.split('/')[1])
    
    second_start = int(second_move.split('/')[0])
    second_end = int(second_move.split('/')[1])
    
    return [first_start, first_end, second_start, second_end]

def adjust_board_for_move(position, pieces):
    'ensure pieces always go down in indices and up in checkers'
    if pieces not in (['x','o']):
        print('Not a legal position')
        return
        
    if pieces == 'x':
        return np.flip(position)
    if pieces == 'o':
        return position * -1
    
def update_board(move,position,pieces='x'):
    #initialize board orientation
    adjust_board_for_move(position, pieces)

    #decrement the places you're moving from
    position[move[0]] -= 1
    position[move[2]] -= 1

    for new_pos in [move[1], move[3]]:
        if position[new_pos] == -1:
            position[new_pos]
        else:
            position[new_pos] += 1

    # return board orientation to original position
    adjust_board_for_move(position, pieces)

    return position
    
def move(position, roll, move, pieces='x'):
    
    move = parse_move(move)
    #check if it fits the roll
    diff1= abs(move[1] - move[0])
    diff2= abs(move[3] - move[2])
    if sorted(list(roll)) != sorted([diff1, diff2]):
        print('This is not a legal move because of dice')
        return
        
    #adjust values for indices
    move = [x-1 for x in move]
    position = adjust_board_for_move(position, pieces)

    #check where you have checkers and where you can move them
    legal_starts = np.where(position > 0)[0]
    open_spaces = np.where(position >= -1)[0]
    # print(f'legal starts : {legal_starts}')
    # print(set([move[1],move[3]]))
    # print(f'open spaces : {open_spaces}')
    # print(set([move[0],move[2]]))

    # confirm these match
    if (set([move[0],move[2]]).issubset(legal_starts)) & set([move[1],move[3]]).issubset(open_spaces):
        new_position = update_board(move,position,pieces)
        # return board orientation to original position
        new_position = adjust_board_for_move(new_position, pieces)
        return new_position
    else:
        print('This is not a legal move2')
        return

## Try rolling 3/1 from the initial board

In [37]:
initial_board

array([ 2,  0,  0,  0,  0, -5,  0, -3,  0,  0,  0,  5, -5,  0,  0,  0,  2,
        0,  4,  2,  0,  0,  0, -2])

In [64]:
initial_board = create_initial_board()
new_position = move(position=initial_board, 
                    roll=[1,3], 
                    move='6/5 8/5', 
                    pieces='x')

[ 2  0  0  0  0 -5  0 -3  0  0  0  5 -5  0  0  0  3  0  5  0  0  0  0 -2]
legal starts : [ 5  7 12 23]
{4}
open spaces : [ 1  2  3  4  5  6  7  8  9 10 12 13 14 15 17 19 20 21 22 23]
{5, 7}


In [65]:
draw_board(new_position)


     |---------------------|
  12 |XXXXX     |     OOOOO|13
  11 |          |          |14
  10 |          |          |15
   9 |          |          |16
   8 |OOO       |        XX|17
   7 |          |          |18
     |---------------------|
     |---------------------|
   6 |OOOOO     |      XXXX|19
   5 |          |        XX|20
   4 |          |          |21
   3 |          |          |22
   2 |          |          |23
   1 |XX        |        OO|24
     |---------------------|

    X remaining pips : 148
    O remaining pips : 152
    


In [66]:
new_position2 = move(position=new_position, 
                    roll=[4,2], 
                    move='8/4 6/4', 
                    pieces='o')

[ 2  0  0  0  0 -5  0 -3  0  0  0  5 -5  0  0  0  2  0  4  2  0  0  0 -2]
legal starts : [ 5  7 12 23]
{3}
open spaces : [ 1  2  3  4  5  6  7  8  9 10 12 13 14 15 17 20 21 22 23]
{5, 7}


In [67]:
draw_board(new_position2)


     |---------------------|
  12 |XXXXX     |     OOOOO|13
  11 |          |          |14
  10 |          |          |15
   9 |          |          |16
   8 |OO        |        XX|17
   7 |          |          |18
     |---------------------|
     |---------------------|
   6 |OOOO      |      XXXX|19
   5 |          |        XX|20
   4 |OO        |          |21
   3 |          |          |22
   2 |          |          |23
   1 |XX        |        OO|24
     |---------------------|

    X remaining pips : 148
    O remaining pips : 146
    


## TODO
- somehow initial board gets updated during this process as well
- doesnt properly handle moving the same piece twice
- [ ] add the ability to do that
- [ ] add the ability to write the above like '24/13'